# <center>NBFI Vehicle Loan Pipelining</center>

In [1]:
#ty
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder, RobustScaler
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_column', None)

C:\Users\hp\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# ty

# spliting the dataset into test and split                                   done
# Data cleaning the dataset, some columns are not in the proper format       done
# separate the categorical and numerical features                            done
# Data preprocessing for caegorical and numerical data                        done
# combining the Data preprocessing process for both                           done

In [5]:
# splittying the dataset into test and split

df = pd.read_csv('Train_Dataset.csv', index_col='ID')
X = df.drop('Default', axis=1)
y = df['Default']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)



class X_cleaner_Transformer(BaseEstimator, TransformerMixin):
    num_col = ['Client_Income','Car_Owned','Bike_Owned','Active_Loan','House_Own','Child_Count','Credit_Amount' ,'Loan_Annuity','Population_Region_Relative','Age_Days','Employed_Days','Registration_Days' ,'ID_Days','Own_House_Age','Mobile_Tag','Homephone_Tag','Workphone_Working','Client_Family_Members' ,'Cleint_City_Rating','Application_Process_Day','Application_Process_Hour','Score_Source_1' ,'Score_Source_2','Score_Source_3','Social_Circle_Default','Phone_Change','Credit_Bureau']

    def _init_(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X_cleaner_Transformer.clean(X)
    
    @staticmethod
    def correct_format(X_df):
        for col in X_cleaner_Transformer.num_col:
            X_df[col] = X_df[col].replace('[a-zA-Z!@#$%^&*()_+=-{};:"<>,./?]', '', regex=True)
            X_df[col] = pd.to_numeric(X_df[col], errors='coerce')
        return X_df

    @staticmethod
    def clean(X_df):
        X_df = X_cleaner_Transformer.correct_format(X_df)
        X_df.drop(columns=['Score_Source_1','Score_Source_3','Social_Circle_Default','Client_Occupation'], inplace=True)
        return X_df
    

    # separate the categorical and numerical features
def categorical_cols(df):
    categorical_features = df.select_dtypes(include=['object'])
    return categorical_features.columns

def numerical_cols(df):
    numerical_features = df.select_dtypes(include=['int64', 'float64'])
    return numerical_features.columns

    

X_train = X_cleaner_Transformer().transform(X_train)
X_test = X_cleaner_Transformer().transform(X_test)


cat_cols = categorical_cols(X_train)
num_cols = numerical_cols(X_train)



In [11]:
# Define preprocessing for numerical data

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler())
])

# Define preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [12]:
# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        #('clean_format', X_transformer, X_train.columns)
        #('sep_cat':separate_num_columns)
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols),
    ])

In [13]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', RobustScaler())]),
                                 Index(['Client_Income', 'Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own',
       'Child_Count', 'Credit_Amount', 'Loan_Annuity',
       'Population_Region_Relative', 'Age_Days', 'Employed_Days',
       'Registration_Days', 'ID_Days', 'Own_House_Age', 'Mobile_Tag',
       'Home...
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index(['Accompany_Client', 'Client_Income_Type', 'Client_Education',
       'Client_Marital_Status', 'Client_Gender', 'Loan_Contract_Type',
       'Client_Housing_Type', 'Client_Permanent_Match_Tag',
       'Client_Contact_Work_Tag', 'Type_Organization'],
      dtype='object'))])

In [14]:
# Define the model
model = RandomForestClassifier()

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

In [15]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['Client_Income', 'Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own',
       'Child_Count', 'Credit_Amount', 'Loan_Annuity',
       'Population_Region_Relative', 'Age_Days', 'Employed_Days',
       'Registration_Days', 'ID_Days', 'O...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Accompany_Client', 'Client_Income_Type', 'Client_Education',
       'Client_Marital_Status', 'Client_Gender', 'Loan_Contract_Type',
       'Client_Housing_Type', 'Client_Permanent_Match_Tag',
       'Client_Contact_Work_Tag', 'Type_Organization'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [16]:
# Train the model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['Client_Income', 'Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own',
       'Child_Count', 'Credit_Amount', 'Loan_Annuity',
       'Population_Region_Relative', 'Age_Days', 'Employed_Days',
       'Registration_Days', 'ID_Days', 'O...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Accompany_Client', 'Client_Income_Type', 'Client_Education',
       'Client_Marital_Status', 'Client_Gender', 'Loan_Contract_Type',
       'Client_Housing_Type', 'Client_Permanent_Match_Tag',
       'Client_Contact_Work_Tag', 'Type_Organization'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [18]:
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

In [19]:
y_pred_test

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [20]:
y_pred_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
# Evaluate the model
print("Training Data Evaluation:")
print(classification_report(y_train, y_pred_train))
print(confusion_matrix(y_train, y_pred_train))
print("Accuracy:", accuracy_score(y_train, y_pred_train))
print("ROC AUC Score:", roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]))

Training Data Evaluation:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     89608
           1       1.00      1.00      1.00      7876

    accuracy                           1.00     97484
   macro avg       1.00      1.00      1.00     97484
weighted avg       1.00      1.00      1.00     97484

[[89608     0]
 [    2  7874]]
Accuracy: 0.9999794838127283
ROC AUC Score: 0.737123569593757


In [22]:
print("\nTest Data Evaluation:")
print(classification_report(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("ROC AUC Score:", roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]))


Test Data Evaluation:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     22403
           1       1.00      0.07      0.14      1969

    accuracy                           0.93     24372
   macro avg       0.96      0.54      0.55     24372
weighted avg       0.93      0.93      0.89     24372

[[22403     0]
 [ 1823   146]]
Accuracy: 0.9252010503856885
ROC AUC Score: 0.737123569593757


In [25]:
# using dill library
import dill

In [26]:
dill.dump(pipeline, open('nbfi_classification_pipeline_dill', 'wb'))
nfbi_model = dill.load(open('nbfi_classification_pipeline_dill','rb'))

In [27]:
nfbi_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['Client_Income', 'Car_Owned', 'Bike_Owned', 'Active_Loan', 'House_Own',
       'Child_Count', 'Credit_Amount', 'Loan_Annuity',
       'Population_Region_Relative', 'Age_Days', 'Employed_Days',
       'Registration_Days', 'ID_Days', 'O...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Accompany_Client', 'Client_Income_Type', 'Client_Education',
       'Client_Marital_Status', 'Client_Gender', 'Loan_Contract_Type',
       'Client_Housing_Type', 'Client_Permanent_Match_Tag',
       'Client_Contact_Work_Tag', 'Type_Organization'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [28]:
print(type(nfbi_model))

<class 'sklearn.pipeline.Pipeline'>


In [29]:
nfbi_model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)